# Cài đặt bài toán vận tải

## 0. Setup

In [408]:
# /*==========================================================================================*\
# **                        _           _ _   _     _  _         _                            **
# **                       | |__  _   _/ | |_| |__ | || |  _ __ | |__                         **
# **                       | '_ \| | | | | __| '_ \| || |_| '_ \| '_ \                        **
# **                       | |_) | |_| | | |_| | | |__   _| | | | | | |                       **
# **                       |_.__/ \__,_|_|\__|_| |_|  |_| |_| |_|_| |_|                       **
# \*==========================================================================================*/


# Author: Group 3
#  * Lê Nguyên Bách 
#  * Hoàng Minh Đức 
#  * Vũ Minh Hiếu
#  * Nguyễn Thị Hường 
#  * Nguyễn Đức Quân 
#  * Bùi Tiến Thành 
# 
# Fabricated by Bùi Tiến Thành (@bu1th4nh)
# Date: 2022/06/27 12:27 AM
# CTTN Toán tin K64

import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import queue
import warnings
import webbrowser as brws
import random


## 1. Một số hàm bổ trợ và debug

In [409]:
def PrintG(G, m, n, itr_id = 0):
#{
    Cinderella = np.zeros((m, n));
    for (u, v, x) in G:
        Cinderella[u, v] = x;

    print("X-matrix at iteration #{0} is:".format(itr_id));
    print(Cinderella);
    print("with cost value ", np.dot(np.reshape(Cinderella, m*n), np.reshape(C, m*n)));
#}
def PrintG(G, m, n, cost, itr_id = 0):
#{
    Cinderella = np.zeros((m, n));
    for (u, v, x) in G:
        Cinderella[u, v] = x;

    print("X-matrix at iteration #{0} is:".format(itr_id));
    print(Cinderella);
    print("with cost value ", np.dot(np.reshape(Cinderella, m*n), np.reshape(cost, m*n)));
#}
def PrintGList(G):
#{
    ret = "";
    for (u, v, x) in G:
        if(len(ret) != 0): ret += " ";
        ret += "({0}, {1})".format(u+1, v+1);
    print(ret);
#}
def PrintDict(dictt):
#{
    for (u, v) in dictt.items():
        print(u, "-->", v);
#}
def PrintCycle(cycle, vtx_mark):
#{
    ret = "Cycle:";
    for i in range(len(cycle)-1):
        (u, v) = cycle[i];
        if(vtx_mark[(u, v)] == 0): continue;
        if(len(ret) != 0): ret += " ";
        ret += "({0}, {1})".format(u+1, v+1);
    print(ret);
#}
def PrintInitialCycle(cycle):
#{
    ret = "Initial Cycle:";
    for i in range(len(cycle)-1):
        (u, v) = cycle[i];
        if(len(ret) != 0): ret += " ";
        ret += "({0}, {1})".format(u+1, v+1);
    print(ret);
#}
def ZeroToOneIndex(id):
#{
    (u, v) = id;
    return (u+1, v+1);
#}
        



def PrintOptimalSolution(G, C, m, n, unbalanced_flag):
#{
    Cinderella = np.zeros((m, n));
    # print(m, n);

    for (u, v, x) in G:
        Cinderella[u, v] = x;

    print("Nghiệm tối ưu:");

    if(unbalanced_flag == "none"):
        print(Cinderella);
    elif(unbalanced_flag == "row"):
        print(Cinderella[0:m-1,0:n]);
    elif(unbalanced_flag == "col"):
        print(Cinderella[0:m,0:n-1]);
        
    print("với giá trị tối ưu là ", np.dot(np.reshape(Cinderella, m*n), np.reshape(C, m*n)));
#}
def GListToMatrix(G, m, n):
#{
    Cinderella = np.zeros((m, n));
    for (u, v, x) in G:
        Cinderella[u, v] = x;
    return Cinderella;
#}

## 2. Xử lý dữ liệu: Các bài toán mở rộng

### 2.1. Bài toán không cân bằng thu phát

In [410]:
def UnbalancedCase(__src, __sink, __cost):
#{
    src = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);
    (m, n) = (len(src), len(sink));
    delta = np.abs(np.linalg.norm(src, 1) - np.linalg.norm(sink, 1));
    if(np.linalg.norm(src, 1) > np.linalg.norm(sink, 1)):
        sink = np.append(sink, delta);
        cost = np.append(cost, np.zeros((m, 1)), axis = 1);
        return "col", src, sink, cost;
    else:
        src  = np.append(src, delta);
        cost = np.append(cost, np.zeros((1, n)), axis = 0);
        return "row", src, sink, cost;
#}

### 2.2. Bài toán ô cấm

In [411]:
def ProhibitedCase(__src, __sink, __cost):
#{
    src  = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);

    (m, n) = (len(src), len(sink));
    pseudo_inf = np.nanmax(cost[cost != np.inf]) * m * n;
    for i in range(m):
        for j in range(n):
            if(np.isnan(cost[i, j]) or np.isinf(cost[i, j])):
                # Ta đưa giá trị này thành giả vô cực cho dễ tính toán
                cost[i, j] = pseudo_inf;
    return (src, sink, cost, pseudo_inf);
#}

### 2.3. Bài toán lập kho nhận hàng

In [412]:
def StorageCase(__src, __sink, __cost):
#{
    src  = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);

    (m, n) = cost.shape;
    sink = np.append(
        arr = sink, 
        values = np.ones(n - len(sink)) * np.linalg.norm(src, 1)
    );

    return (src, sink, cost)
#}

## 3. Tìm phương án cực biên xuất phát

### 3.1. Thủ tục cập nhật bảng

In [413]:
def UpdateTable(src, sink, cost, u, v):
#{
    (m, n) = (len(src), len(sink));
    inf = (np.max(src) * (m+1)) + (np.max(sink) * (n+1)) * np.nanmax(cost[cost != np.inf]) * 69;

    rtn_amt = min(src[u], sink[v]);
    is_row_remain = False;
    if(src[u] < sink[v]):
        # Update
        sink[v] -= rtn_amt;
        src[u]   = 0;
        # Delete row
        cost[u]  = np.ones(n) * inf;
        is_row_remain = False;
    else:
        # Update
        src[u]  -= rtn_amt;
        sink[v]  = 0;
        # Delete column
        cost[:, v] = np.ones(m).transpose() * inf;
        is_row_remain = True;
    return (rtn_amt, is_row_remain);
#}



### 3.2. Phương pháp góc tây bắc

In [414]:

def NorthWestMethod(__src, __sink, __cost):
#{
    # Phương pháp góc tây bắc
    src = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);
    (m, n) = (len(src), len(sink));
    G = list(); # G: vector<tuple<int, int, double>> 

    qx = queue.PriorityQueue();
    qy = queue.PriorityQueue();

    for i in range(m): qx.put(i);
    for i in range(n): qy.put(i);

    while(len(G) < m + n - 1):
        u = qx.get();
        v = qy.get();
        (rtn_amt, irr) = UpdateTable(src, sink, cost, u, v);
        G.append((u, v, rtn_amt));
        if(irr): qx.put(u);
        else: qy.put(v);
    return G;
#}



### 3.3. Phương pháp cực tiểu chi phí

In [415]:

def MinCostMethod(__src, __sink, __cost):
#{
    # Phương pháp cực tiểu chi phí
    src = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);
    (m, n) = (len(src), len(sink));
    G = list(); # G: vector<tuple<int, int, double>> 

    while(len(G) < m + n - 1):
        gg = np.unravel_index(np.nanargmin(cost, axis=None), cost.shape);
        u = gg[0];
        v = gg[1];
        (rtn_amt, _) = UpdateTable(src, sink, cost, u, v);
        G.append((u, v, rtn_amt));
        
        # print("A = ", src)
        # print("B = ", sink)
        # print("C = \n", cost)  
        # print(G)  
        # print(X)  

    return G;
#}



### 3.4. Phương pháp Vogel

In [416]:

def VogelApproxMethod(__src, __sink, __cost):
#{
    # Phương pháp Vogel
    src = np.copy(__src);
    sink = np.copy(__sink);
    cost = np.copy(__cost);
    (m, n) = (len(src), len(sink));
    G = list(); # G: vector<tuple<int, int, double>> 
    inf = (np.max(src) * (m+1)) + (np.max(sink) * (n+1)) * np.nanmax(cost[cost != np.inf]) * 69;

    # Initialize penalty row/cols
    costRow = np.zeros(m);
    costCol = np.zeros(n);
        

    # VAM Method
    while(len(G) < m + n - 1):
        # print("Iteration #", len(G))    
        # print("A = ", src)
        # print("B = ", sink)
        # print("C = \n", cost)  


        # Partial sort the cost array by cols and rows
        Ariel = np.partition(cost, (0,1), axis = 0);
        Belle = np.partition(cost, (0,1), axis = 1);

        # Compute penalty value -- Fark, we can't use vectorization here
        for i in range(n):
            if(Ariel[0, i] != inf):
                if(Ariel[1, i] != inf): costCol[i] = Ariel[1, i] - Ariel[0, i];
                else: costCol[i] = 0;
            else: costCol[i] = -np.inf;

        for i in range(m):
            if(Belle[i, 0] != inf):
                if(Belle[i, 1] != inf): costRow[i] = Belle[i, 1] - Belle[i, 0];
                else: costRow[i] = 0;
            else: costRow[i] = -np.inf;

        # print("Penalty: R", costRow);
        # print("Penalty: C", costCol);
        
        # Find cols/row has largest penalty value
        ### List of row indices has largest penalty value
        Ariel = np.array(range(m))[costRow == np.max(costRow)]; 
        ### List of column indices has largest penalty value
        Belle = np.array(range(n))[costCol == np.max(costCol)];

        # Choose the minimum cost
        u = -1;
        v = -1;
        opt_cost = np.inf;

        if(np.max(costRow) >= np.max(costCol)):
            for i in Ariel:
                if(
                    (u == -1 and v == -1) or
                    (opt_cost < np.min(cost[i, :]))
                ): (opt_cost, u, v) = (np.min(cost[i, :]), i, np.argmin(cost[i, :]));
        
        if(np.max(costRow) <= np.max(costCol)):
            for j in Belle:
                if(
                    (u == -1 and v == -1) or
                    (opt_cost < np.min(cost[:, j]))
                ): (opt_cost, u, v) = (np.min(cost[:, j]), np.argmin(cost[:, j]), j);

        # Update table
        (rtn_amt, _) = UpdateTable(src, sink, cost, u, v);
        G.append((u, v, rtn_amt));


        # print(Ariel)  
        # print(Belle)  
        # print(G)  
        # print(X)  
        # print()
        # print()


    return G;
#}



## 4. Thuật toán chính

In [417]:
def PotentialMethod(
    Supply,                         # Lượng phát
    Demand,                         # Lượng thu
    Cost,                           # Chi phí
    G                               # PACB xuất phát
):
#{
    (m, n) = (len(Supply), len(Demand));
    (src, sink, cost) = (np.copy(Supply), np.copy(Demand), np.copy(Cost));

    
    nIteration = 0;
    while(True):        
    #{
        ######################################################################
        # Setup, chuyển dữ liệu ra mảng X
        ######################################################################
        nIteration += 1;
        X = dict();
        for (u, v, x) in G: X[(u, v)] = x;




        # Debug
        # print()
        # print()
        # print()
        # PrintG(G, nIteration);
        ######################################################################
        # Xác định thế vị $u_i, v_j$ bằng giải HPT
        # Encoding: First m indices are m(row), last n indices are n(columns)
        ######################################################################
        Ariel = np.zeros((m+n, m+n));
        belle = np.zeros((m+n, 1));
        

        (u, _, _)         = G[0];
        Ariel[m+n-1, u]   = 1;
        belle[m+n-1]      = 0;
        for ith_coordinate in range(len(G)):
        #{
            (u, v, _)                  = G[ith_coordinate];
            Ariel[ith_coordinate, u]   = 1;
            Ariel[ith_coordinate, m+v] = 1;
            belle[ith_coordinate]      = cost[u, v];
        #}
        # print(Ariel);
        # print(np.linalg.det(Ariel));
        assert np.linalg.det(Ariel) != 0;
        UV = np.linalg.inv(Ariel) @ belle;

        


        ######################################################################
        # Tính ước lượng $\Delta_{ij}$. Các ô $(i, j) \in G(x)$ sẽ được gán
        # là -inf để không ảnh hưởng đến phép kiểm tra ở bước sau
        ######################################################################
        Delta = np.zeros((m, n));
        for (i, j, _) in G: Delta[i, j] = -np.inf;
        for i in range(m):
        #{
            for j in range(n):
                if(Delta[i, j] != -np.inf):
                    Delta[i, j] = UV[i] + UV[m+j] - cost[i, j];
        #}  
                    



        # print("Delta: ");
        # print(Delta);
        ######################################################################
        # Kiểm tra điều kiện dừng là chặn dưới của $\Delta_{ij}$.
        ######################################################################
        if(np.max(Delta, axis=None) <= 0):
        #{
            return (G, nIteration);
            # PrintOptimalSolution(G);
            # if(random.randint(0, 1) == 0): 
            #     brws.open("https://www.youtube.com/watch?v=3Nz2Tf9Drp8");
            # else: 
            #     brws.open("https://www.youtube.com/watch?v=dQw4w9WgXcQ");
            # print();
            # print();
            # print();
        #}
            


        ######################################################################
        # Xác định ô điều chỉnh $(i_s, j_s)$ và thêm nó vào G
        ######################################################################
        (i_s, j_s) = np.unravel_index(np.argmax(Delta, axis=None), Delta.shape);
        G.append((i_s, j_s, -1));





        ######################################################################
        # Tìm và khởi tạo đánh dấu chu trình điều chỉnh  
        ######################################################################
        # Xây dựng đồ thị
        row_map  = dict();  # Đánh dấu những ô nào nằm hàng i
        col_map  = dict();  # Đánh dấu những ô nào nằm cột j
        adj_list = dict();  # Danh sách kề chứa đỉnh
        vtx_mark = dict();  # Mảng đánh dấu +/-/0 như trong algo. Nếu chưa xét thì đỉnh đó đánh dấu 404 (Not Found)
        for (u, v, _) in G:
        #{
            row_map[u] = list();
            col_map[v] = list();
        #}
        for (u, v, _) in G:
        #{
            row_map[u].append(v)
            col_map[v].append(u)
        #}
        for (u, v, _) in G:
        #{
            adj_list[(u, v)] = list();
            vtx_mark[(u, v)] = 404;  
        #}
        for u, v_list in row_map.items():
        #{
            v_list = sorted(v_list);
            for i in range(len(v_list) - 1):
                adj_list[(u, v_list[i]  )].append((u, v_list[i+1]));
                adj_list[(u, v_list[i+1])].append((u, v_list[i]  ));
        #}
        for v, u_list in col_map.items():
        #{
            u_list = sorted(u_list);
            for i in range(len(u_list) - 1):
                adj_list[(u_list[i],   v)].append((u_list[i+1], v));
                adj_list[(u_list[i+1], v)].append((u_list[i],   v));
        #}
        



        ######################################################################
        # Đánh dấu chu trình điều chỉnh  
        ######################################################################
        # Duyệt trước các đỉnh có bậc 1
        Degree_1 = list(); # Tập các đỉnh có bậc 1
        for u in adj_list:
        #{
            if(len(adj_list[u]) == 1): Degree_1.append(u);
        #}
        for u in Degree_1: # Tìm đường đến đỉnh bậc 3
        #{
            vtx_mark[u] = 0;
            while(len(adj_list[u]) != 3):
                for v in adj_list[u]:
                    if(vtx_mark[v] != 404): continue;
                    u = v;
                    break; # Ta biết mỗi đỉnh trong đồ thị này có bậc không quá 3, nếu là 3 thì break
                if(len(adj_list[u]) < 3): vtx_mark[u] = 0;
        #}




        # Tìm chu trình: Khởi tạo
        cycle                = list(); 
        vtx_queue            = queue.LifoQueue();
        start_vtx            = (i_s, j_s);
        X[start_vtx]         = 0;
        vtx_mark[start_vtx]  = 1;
        vtx_queue.put(start_vtx);
        cycle.append(start_vtx);




        # Engineering Tweak: Chỉ cho đỉnh xuất phát 1 đường ra duy nhất
        start_vtx = (i_s, j_s);
        while(len(adj_list[start_vtx]) > 1): adj_list[start_vtx].pop();



        # Tìm chu trình: DFS
        while(not vtx_queue.empty()):
        #{
            u = vtx_queue.get();
            # print(ZeroToOneIndex(u), vtx_mark[u]);
            for v in adj_list[u]: 
                # print("----------", ZeroToOneIndex(v), vtx_mark[v]);
                if(vtx_mark[v] == 404): 
                    vtx_mark[v] = 1;
                    vtx_queue.put(v);
                    cycle.append(v);
        #}
        cycle.append(start_vtx);




        # Tìm chu trình: Đánh dấu và thu gọn chu trình
        for i in range(1, len(cycle)-1):
        #{
            (u_last, v_last) = cycle[i-1];
            (u     , v     ) = cycle[i];
            (u_next, v_next) = cycle[i+1];
            if(
                (u_last == u and u == u_next) or
                (v_last == v and v == v_next)
            ): vtx_mark[(u, v)] = 0;
            else: 
                j = i-1;
                while(vtx_mark[cycle[j]] == 0): j -= 1; # Ta không cần kiểm tra ĐK j = 0 vì chắc chắn ở j = 0 thì cycle[j] có nhãn là '+', hay 1
                vtx_mark[(u, v)] = -1 * vtx_mark[cycle[j]];
        #}


        




        # PrintInitialCycle(cycle);
        # PrintCycle(cycle, vtx_mark);
        # print("i_s, j_s =", (i_s+1, j_s+1));
        ######################################################################
        # Xây dựng phương án mới
        ######################################################################
        # Tìm theta
        (i_r, j_r, theta) = (-1, -1, np.nan);
        for (u, mark) in vtx_mark.items():
        #{
            if(mark == -1):
                if(np.isnan(theta)): 
                    theta = X[u];
                    (i_r, j_r) = u;
                elif(theta > X[u]):
                    theta = X[u];
                    (i_r, j_r) = u;
                elif(theta == X[u]):
                    if(cost[i_r, j_r] < cost[u]): (i_r, j_r) = u;
        #}







        # print("Theta =", theta);
        # print("i_r, j_r =", (i_r+1, j_r+1));
        # Cập nhật G và X
        new_G = list();
        for (u, v, x) in G:
        #{
            if(u == i_r and v == j_r):    continue;
            elif((u, v) == start_vtx):    new_G.append((u, v, theta));
            elif(vtx_mark[(u, v)] ==  1): new_G.append((u, v, x + theta));
            elif(vtx_mark[(u, v)] == -1): new_G.append((u, v, x - theta));
            elif(vtx_mark[(u, v)] ==  0): new_G.append((u, v, x));
        #}
        G.clear();
        G = new_G;
    #} 
#}



## 5. Solver

In [418]:
def TransportProblemSolver(
    supply                       : np.array,
    demand                       : np.array,
    cost                         : np.ndarray,
    prohibited_detection_method  = "autodetect",
    manual_prohibited_list       = list(),
    initial_cfg_method           = "manual",
    manual_init_cfg_input_method = "list",
    manual_init_cfg              = list()

):
#{
    (m, n) = (len(supply), len(demand));

    # ----------------------------------------------------------------------------------
    # Trường hợp lập kho nhận hàng
    storage_flag = False;
    n_old        = n;
    if(cost.shape[1] > n):
    #{
        storage_flag = True;
        (supply, demand, cost) = StorageCase(supply, demand, cost);
        (m, n) = (len(supply), len(demand));
    #}
        



    # ----------------------------------------------------------------------------------
    # Dò và xử lý ô cấm
    prohibited_flag = False;
    if(prohibited_detection_method == "manual"):
        for u in manual_prohibited_list:
            cost[u] = np.inf;
    if(np.isinf(cost).any() or np.isnan(cost).any()):
    #{
        prohibited_flag = True;
        (supply, demand, cost, pseudo_inf) = ProhibitedCase(supply, demand, cost);
        # print(cost);
    #}

    

    # ----------------------------------------------------------------------------------
    # Xử lý không cân bằng thu phát
    unbalanced_flag = "none";
    if(np.linalg.norm(supply, 1) != np.linalg.norm(demand, 1)):
    #{
        (unbalanced_flag, supply, demand, cost) = UnbalancedCase(supply, demand, cost);
        (m, n) = (len(supply), len(demand));
    #}



    # ----------------------------------------------------------------------------------
    # Chọn PACB xuất phát
    G = list();
    if(initial_cfg_method == "manual"):
    #{
        if(manual_init_cfg_input_method == "matrix"):
            manual_init_cfg = np.array(manual_init_cfg);
            for i in range(m):
                for j in range(n):
                    if(manual_init_cfg[i, j] != 0):
                        G.append((i, j, manual_init_cfg[i, j]));
        elif(manual_init_cfg_input_method == "list"):
            G = manual_init_cfg;
        else: raise NotImplementedError("This method is not yet implemented");
        # PrintG(G, m, n, 0);
    #}
    elif(initial_cfg_method == "northwest"):
        G = NorthWestMethod(supply, demand, cost);
        # PrintG(G, m, n, 0);
    elif(initial_cfg_method == "mincost"):
        G = MinCostMethod(supply, demand, cost);
        # PrintG(G, m, n, 0);
    elif(initial_cfg_method == "vogel"):
        G = VogelApproxMethod(supply, demand, cost);
        # PrintG(G, m, n, 0);
    else: raise NotImplementedError("This method is not yet implemented");



    # ----------------------------------------------------------------------------------
    # Giải!
    (G, nIter) = PotentialMethod(supply, demand, cost, G);



    # ----------------------------------------------------------------------------------
    # Kiểm tra flag
    print("==================== TÍNH CHẤT BÀI TOÁN ====================");

    # Trường hợp ô cấm
    if(prohibited_flag):
        print("Bài toán này có ô cấm");
    if(prohibited_flag):
    #{
        sol_matrix = GListToMatrix(G, m, n);
        if(np.logical_and((sol_matrix != 0), (cost == pseudo_inf)).any()):
        #{
            print();
            print("===================== KẾT QUẢ BÀI TOÁN =====================");
            print("Bài toán không có nghiệm tối ưu, nghiệm BT mở rộng tìm ra sau {0} bước lặp".format(nIter));
            print("============================================================");
            print()
            print()
            return;
        #}
    #}
        
    # Trường hợp BTVT không cân
    if(unbalanced_flag != "none"):
    #{
        print("Bài toán này không cân bằng thu phát");
        if(unbalanced_flag == "col"):
            G_new = list();
            for (u, v, x) in G:  
                if(v == n): continue;
                G_new.append((u, v, x));
        else:
            G_new = list();
            for (u, v, x) in G:  
                if(u == m): continue;
                G_new.append((u, v, x));
        G = G_new;
    #}

    # Trường hợp lập kho nhận hàng
    if(storage_flag): print("Bài này cần lập kho nhận hàng");

    # Trường hợp cơ bản
    if(
        (not storage_flag)          and
        (not prohibited_flag)       and
        (unbalanced_flag == "none")
    ): print("Bài toán vận tải kơ bản");
    
    

    # ----------------------------------------------------------------------------------
    # In kết quả
    # In ra kho hàng nếu có
    print("===================== KẾT QUẢ BÀI TOÁN =====================");
    if(storage_flag):
    #{
        print("Chi tiết kho hàng thêm:");
        Ariel = GListToMatrix(G, m, n);
        for i in range(n_old, n):
            desired_capacity = np.linalg.norm(Ariel[0:m-1, i], 1);
            if(desired_capacity == 0):
                print("Không cần lập kho nhận hàng số {0}".format(i+1));
            else:
                print("Kho nhận hàng số {0} cần có trữ lượng hàng là {1} đơn vị".format(i+1, desired_capacity));
        print("----------");  
        print();  
    #}
      

    # In ra nghiệm
    print("Bài toán có nghiệm tối ưu sau {0} lần lặp".format(nIter));
    PrintOptimalSolution(G, cost, m, n, unbalanced_flag);
    if(random.randint(0, 1) == 0): 
        brws.open("https://www.youtube.com/watch?v=3Nz2Tf9Drp8");
    else: 
        brws.open("https://www.youtube.com/watch?v=dQw4w9WgXcQ");
    print("============================================================");
    print()
    print()
    
    
#}

## 6. Chạy ví dụ

### 6.1. Trường hợp cơ bản

In [419]:

# # Vector đỉnh phát 
# a = np.array([60, 70, 20], dtype=float);
# # Vector đỉnh thu
# b = np.array([30, 40, 30, 50], dtype=float);
# # Ma trận chi phí
# C = np.array([
#     [1, 2, 4, 3],
#     [2, 3, 2, 7],
#     [3, 5, 6, 4]
# ], dtype=float);
# m   = len(a);
# n   = len(b);
# Vector đỉnh phát 
a = np.array([10, 30, 20], dtype=float);
# Vector đỉnh thu
b = np.array([25, 25, 10], dtype=float);
# Ma trận chi phí
C = np.array([
    [5, 3, 1],
    [7, 6, 8],
    [3, 2, 2]
], dtype=float);
m   = len(a);
n   = len(b);



# TransportProblemSolver(
#     supply = a, 
#     demand = b, 
#     cost = C, 
#     prohibited_detection_method ="autodetect",
#     initial_cfg_method ="manual",
#     manual_init_cfg_input_method ="list",
#     manual_init_cfg = [(0, 0, 30), (0, 1, 30), (1, 1, 10), (1, 2, 30), (1, 3, 30), (2, 3, 20)]
# );

TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="northwest"
);

TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="mincost"
);

TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="vogel"
);



==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 2 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.]
 [25.  5.  0.]
 [ 0. 20.  0.]]
với giá trị tối ưu là  255.0


==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.]
 [25.  5.  0.]
 [ 0. 20.  0.]]
với giá trị tối ưu là  255.0


==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.]
 [ 5. 25.  0.]
 [20.  0.  0.]]
với giá trị tối ưu là  255.0




### 6.2. Trường hợp có ô cấm

In [420]:
# Vector đỉnh phát 
a = np.array([10, 40], dtype=float);
# Vector đỉnh thu
b = np.array([10, 20, 10, 10], dtype=float);
# Ma trận chi phí
C = np.array([
    [np.inf, 5, 5, np.nan],
    [5, 2, 4, 3]
], dtype=float);



# print("NorthWestMethod:")
# PrintG(NorthWestMethod(a, b, C));
# print("MinCostMethod:")
# PrintG(MinCostMethod(a, b, C));
# print("VogelApproxMethod:")
# PrintG(VogelApproxMethod(a, b, C));


TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="vogel"
);

==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán này có ô cấm
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.  0.]
 [10. 20.  0. 10.]]
với giá trị tối ưu là  170.0




### 6.3. Trường hợp không cân bằng

In [421]:
# Vector đỉnh phát 
a = np.array([80, 70, 50], dtype=float);
# Vector đỉnh thu
b = np.array([100, 50, 30, 70], dtype=float);
# Ma trận chi phí
C = np.array([
    [3, 5, 6, 5],
    [4, 5, 7, 8],
    [3, 4, 4, 3]
], dtype=float);




TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="northwest"
);

==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán này không cân bằng thu phát
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 3 lần lặp
Nghiệm tối ưu:
[[80.  0.  0.  0.]
 [20. 50.  0.  0.]
 [ 0.  0.  0. 50.]]
với giá trị tối ưu là  720.0




### 6.4. Trường hợp lập kho nhận hàng

In [422]:
# Vector đỉnh phát 
a = np.array([150, 85, 180], dtype=float);
# Vector đỉnh thu
b = np.array([100, 95], dtype=float);
# Ma trận chi phí
C = np.array([
    [3, 5, 2],
    [5, 1, 4],
    [7, 4, 9]
], dtype=float);




TransportProblemSolver(
    supply = a, 
    demand = b, 
    cost = C, 
    prohibited_detection_method ="autodetect",
    initial_cfg_method ="mincost"
);

==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán này không cân bằng thu phát
Bài này cần lập kho nhận hàng
===================== KẾT QUẢ BÀI TOÁN =====================
Chi tiết kho hàng thêm:
Kho nhận hàng số 3 cần có trữ lượng hàng là 235.0 đơn vị
----------

Bài toán có nghiệm tối ưu sau 3 lần lặp
Nghiệm tối ưu:
[[  0.   0. 150.]
 [  0.   0.  85.]
 [ 85.  95.   0.]]
với giá trị tối ưu là  1615.0




## 7. The end

In [423]:
#                 _
#         ~   ,="` `""=,       o
#            /   ,=="""'=;        , __
#    ~      /      ,--'/=,)  o    \`\\"._     _,
#          |   .='/ <9(9.="       / _  |||;._//)
#         /     (J    ^\ \     o_/@ @  ///  |=(
#       .'    .' \  '='/  '-.  ( (`__,     ,`\|
#      /     /    \`-;_      \  '.\_/ |\_.'   
# ~   /      |   /` _  \      )   `""```
#    | ,     ;  /`\/ `\ \    /.-._///_
#    |/     '   \_,\__/\ \.-'.'----'`
#     \|     '.   \     \   /`-,   ~
#       `\     _.-'\    (`-`  .'
#         `-.-' _.-')__./,--'
#      .--'`,-'`'""`    ` \
#     /`"`-`               |          ~
#    |                     /
# ~  |     .-'__         .'
#     \   ;'"`  `""----'`
#      \   \
#       '.  `\
#         )   `-.            ~
#        /       `-._
#       |     ,      `-,
#    ~  \   .' `''----`
#        `.(
#          `
# Fabricated by bu1th4nh

## Bonus: Phương pháp Vogel

In [424]:
# Vector đỉnh phát 
a = np.array([90, 100, 110], dtype=float);
# Vector đỉnh thu
b = np.array([50, 80, 95, 75], dtype=float);
# Ma trận chi phí
C = np.array([
    [11, 2, 16, 3],
    [5, 10, 6, 8],
    [4, 12, 5, 9]
], dtype=float);
m   = len(a);
n   = len(b);



PrintG(NorthWestMethod(a, b, C), m, n, C);
PrintG(MinCostMethod(a, b, C), m, n, C);
PrintG(VogelApproxMethod(a, b, C), m, n, C);



TransportProblemSolver(
    supply=a,
    demand=b,
    cost=C,
    prohibited_detection_method="autodetect",
    initial_cfg_method="mincost"
);
TransportProblemSolver(
    supply=a,
    demand=b,
    cost=C,
    prohibited_detection_method="autodetect",
    initial_cfg_method="vogel"
);


X-matrix at iteration #0 is:
[[50. 40.  0.  0.]
 [ 0. 40. 60.  0.]
 [ 0.  0. 35. 75.]]
with cost value  2240.0
X-matrix at iteration #0 is:
[[ 0. 80.  0. 10.]
 [ 0.  0. 35. 65.]
 [50.  0. 60.  0.]]
with cost value  1420.0
X-matrix at iteration #0 is:
[[ 0. 80.  0. 10.]
 [35.  0.  0. 65.]
 [15.  0. 95.  0.]]
with cost value  1420.0
==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0. 80.  0. 10.]
 [ 0.  0. 35. 65.]
 [50.  0. 60.  0.]]
với giá trị tối ưu là  1420.0


==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0. 80.  0. 10.]
 [35.  0.  0. 65.]
 [15.  0. 95.  0.]]
với giá trị tối ưu là  1420.0




In [425]:
# Vector đỉnh phát 
a = np.array([10, 40], dtype=float);
# Vector đỉnh thu
b = np.array([25, 10, 15], dtype=float);
# Ma trận chi phí
C = np.array([
    [30, 40, 35],
    [55, 90, 130]
], dtype=float);
m   = len(a);
n   = len(b);





print("NW");
PrintG(NorthWestMethod(a, b, C), m, n, C);
TransportProblemSolver(
    supply=a,
    demand=b,
    cost=C,
    prohibited_detection_method="autodetect",
    initial_cfg_method="northwest"
);
print("MC");
PrintG(MinCostMethod(a, b, C), m, n, C);
TransportProblemSolver(
    supply=a,
    demand=b,
    cost=C,
    prohibited_detection_method="autodetect",
    initial_cfg_method="mincost"
);
print("Vogel");
PrintG(VogelApproxMethod(a, b, C), m, n, C);
TransportProblemSolver(
    supply=a,
    demand=b,
    cost=C,
    prohibited_detection_method="autodetect",
    initial_cfg_method="vogel"
);


NW
X-matrix at iteration #0 is:
[[10.  0.  0.]
 [15. 10. 15.]]
with cost value  3975.0
==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 2 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.]
 [25. 10.  5.]]
với giá trị tối ưu là  3275.0


MC
X-matrix at iteration #0 is:
[[10.  0.  0.]
 [15. 10. 15.]]
with cost value  3975.0
==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 2 lần lặp
Nghiệm tối ưu:
[[ 0.  0. 10.]
 [25. 10.  5.]]
với giá trị tối ưu là  3275.0


Vogel
X-matrix at iteration #0 is:
[[ 0.  0. 10.]
 [25. 10.  5.]]
with cost value  3275.0
==================== TÍNH CHẤT BÀI TOÁN ====================
Bài toán vận tải kơ bản
===================== KẾT QUẢ BÀI TOÁN =====================
Bài toán có nghiệm tối ưu sau 1 lần lặp
Nghiệm tối ưu:
[[ 0.  